In [1]:
import tensorflow as tf
import keras
from keras.layers import Conv2D, Conv2DTranspose
import os
import cv2
import numpy as np

In [2]:
conv_kwargs = {
    "padding"             : "SAME",
    "activation"          : keras.layers.LeakyReLU(alpha=0.2),
    "kernel_initializer"  : tf.random_normal_initializer(stddev=.1)
}

In [3]:
class Autoencoder(tf.keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.encoder = keras.Sequential([
            Conv2D(16, 8, 2, **conv_kwargs),
            Conv2D(16, 8, 2, **conv_kwargs),
            keras.layers.MaxPooling2D(),
            Conv2D(64, 4, 1, **conv_kwargs),
        ], name="ae_encoder")

        self.decoder = keras.Sequential([
        Conv2DTranspose(64, 4, 1, **conv_kwargs),
        Conv2DTranspose(16, 8, 2, **conv_kwargs),
        Conv2DTranspose(3, 8, 4, padding='same', kernel_initializer=tf.random_normal_initializer(stddev=.1))
    ], name='ae_decoder')

    def call(self, inputs):
        inputs = self.encoder(inputs)
        inputs = self.decoder(inputs)
        return inputs


In [4]:
def custom_loss(y_true, y_pred):
    mse_loss = keras.losses.MeanSquaredError()
    bce_loss = keras.losses.BinaryCrossentropy()
    mae_loss = keras.losses.MeanAbsoluteError()
    mse = mse_loss(y_true, y_pred)
    #bce = bce_loss(y_true, y_pred)
    mae = mae_loss(y_true, y_pred)
    loss = .3*mse + .7*mae
    return loss

In [5]:
ae_model = Autoencoder(name='autoencoder')

ae_model.build(input_shape = (1,256,256,3))   ## Required to see architecture summary
initial_weights = ae_model.get_weights() ## Just so we can reset out autoencoder

ae_model.summary()
ae_model.encoder.summary()
ae_model.decoder.summary()

ae_model.compile(
    optimizer   = keras.optimizers.legacy.Adam(learning_rate=0.001),
    loss        = custom_loss,
    metrics     = [
        tf.keras.metrics.MeanSquaredError(),
        tf.keras.metrics.BinaryCrossentropy(),
        tf.keras.metrics.MeanAbsoluteError()
    ]
)

Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ae_encoder (Sequential)     (1, 32, 32, 64)           35936     
                                                                 
 ae_decoder (Sequential)     (1, 256, 256, 3)          134227    
                                                                 
Total params: 170163 (664.70 KB)
Trainable params: 170163 (664.70 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: "ae_encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (1, 128, 128, 16)         3088      
                                                                 
 conv2d_1 (Conv2D)           (1, 64, 64, 16)           16400     
                                                             

2024-05-01 19:45:12.451871: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-05-01 19:45:12.451903: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-05-01 19:45:12.451907: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-05-01 19:45:12.451976: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-05-01 19:45:12.452000: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
import sys

isColab = "google.colab" in sys.modules
data_dir = '../collapsed_data'
# this also works:
# isColab = "COLAB_GPU" in os.environ

if isColab:
    from google.colab import drive
    drive.mount("/content/drive", force_remount=True)

    data_dir = ("/content/drive/MyDrive/collapsed_data")

In [7]:
def get_data(sub_dir, size=(256,256)):
    dir = os.path.join(data_dir, sub_dir)
    files = os.listdir(dir)
    x = []
    y = []
    for f in files:
        path = os.path.join(dir, f)
        try:
            img = cv2.imread(path)
            img = cv2.resize(img, size)
            y.append(img)
        except:
            print(path)
            continue
    y = np.array(y, dtype=np.float32)
    x = np.copy(y)
    h,w = size
    rec_w = w//5
    # Make middle black
    x[:,:,rec_w*2:rec_w*3,:]=0
    y=y/255
    x = x/255
    print(x.shape)
    print(y.shape)
    print('done')
    return x,y


In [ ]:
x_train,y_train = get_data('train')
x_valid,y_valid = get_data('validation')

In [ ]:
# Train the model
print('Fitting model')
ae_model.fit(x_train, y_train, epochs=10, batch_size=100, validation_data=(x_valid,y_valid))
#ae_model.fit(x_train, y_train, epochs=5, batch_size=64)
print('------------------------')

In [ ]:
ae_model.save('model.keras')

In [8]:
ae_model = keras.models.load_model('../models/MseMaeWeighted.keras',custom_objects={'Autoencoder': Autoencoder,'custom_loss':custom_loss})

/Users/pratham/anaconda3/lib/python3.11/site-packages/keras/src/initializers/__init__.py:144: UserWarning: The `keras.initializers.serialize()` API should only be used for objects of type `keras.initializers.Initializer`. Found an instance of type <class 'tensorflow.python.ops.init_ops_v2.RandomNormal'>, which may lead to improper serialization.
  warnings.warn(


In [9]:
print('Evaluating model on testing data')
x_test,y_test = get_data('test')
ae_model.evaluate(x_test, y_test, batch_size=32)

Evaluating model on testing data
(500, 256, 256, 3)
(500, 256, 256, 3)
done
 1/16 [>.............................] - ETA: 4s - loss: 0.0527 - mean_squared_error: 0.0105 - mean_absolute_error: 0.0707

2024-05-01 19:45:26.219550: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


16/16 [==============================] - 1s 42ms/step - loss: 0.0524 - mean_squared_error: 0.0106 - mean_absolute_error: 0.0703


[0.052372947335243225, 0.010550432838499546, 0.07029688358306885]

In [11]:
x = x_test[:5]
y = y_test[:5]
pred = ae_model.predict(x)
for i in range(4):
    cv2.imshow('im', x[i])
    cv2.waitKeyEx()
    cv2.imshow('truth', y[i])
    cv2.waitKeyEx()
    cv2.imshow('pred', pred[i])
    cv2.waitKeyEx()
   
cv2.destroyAllWindows()
cv2.waitKey(1)

1/1 [==============================] - 0s 101ms/step


-1

In [15]:
x = x_test[:5]
h,w=(256,256)
rec_w = w//5
modified = []
for i in range(4):
    img1 = x[i]
    img2 = x[i+1]
    new_img = img1.copy()
    new_img[:,rec_w*3:rec_w*5,:]=img2[:,rec_w*3:rec_w*5,:]
    modified.append(new_img)
modified = np.array(modified)
preds = ae_model.predict(modified)
for i in range(4):
    mod_img = modified[i]
    cv2.imshow('mod', mod_img)
    cv2.waitKeyEx()
    cv2.imshow('pred', preds[i])
    cv2.waitKeyEx()

1/1 [==============================] - 0s 134ms/step


IndexError: index 4 is out of bounds for axis 0 with size 4

: 